In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from keras import models
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization

from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix



# split 개수, 셔플 여부 및 seed 설정
str_kf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 14)


Real_result = []
R_Arr_results = []


# label num
Arr_num = -1
model_count = 0



# EarlyStopping
class MyModelCheckpoint(tf.keras.callbacks.ModelCheckpoint):

    def __init__(self, *args, **kwargs):
        super(MyModelCheckpoint, self).__init__(*args, **kwargs)

    # redefine the save so it only activates after 100 epochs
    def on_epoch_end(self, epoch, logs=None):
        if epoch >= 1: super(MyModelCheckpoint, self).on_epoch_end(epoch, logs)



# Arrhythmia classification model architecture
###############################################################################
def build_model_arr():
    inputs = Input(shape=(170, 1), name='main_input') #170: 시계열 데이터의 길이, 1: 채널 수
    x = Conv1D(64, 3, padding='same', activation='relu', strides=2)(inputs)
    x = MaxPooling1D(2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Conv1D(128, 3, padding='same', activation='relu')(x)
    x = MaxPooling1D(2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Conv1D(64, 3, padding='same', activation='relu')(x)
    x = MaxPooling1D(2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Flatten()(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(2, activation='softmax', name='arrhythmia')(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model




# data call
###############################################################################
train_data = np.load('./train_data.npy')
test_data = np.load('./test_data.npy')


# model data dimension expansion
###############################################################################
train_data = np.expand_dims(train_data, axis = -1)
test_data = np.expand_dims(test_data, axis = -1)



# Arrhythmia lable
###############################################################################
train_label = np.zeros([4200, 1])
test_label = np.zeros([1260, 1])



for i in range(len(train_label)) :
    if i >=2100:
        break
    train_label[i]= 1

for i in range(len(test_label)) :
    if i >= 630:
        break
    test_label[i]=1
 
 
 # Arrhythmia train
 ###############################################################################    
for train_index, val_index in str_kf.split(train_data, train_label):
    
    T_data, V_data = train_data[train_index], train_data[val_index]
    T_label, V_label = train_label[train_index], train_label[val_index]
    
    
    # One Hot Encoding Lable
    T_label = tf.keras.utils.to_categorical(T_label)
    V_label = tf.keras.utils.to_categorical(V_label)
    t_label = tf.keras.utils.to_categorical(test_label)

    

    print("R_Arrhythmia")
    model = build_model_arr()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # EarlyStopping 조기종료 및 모델 학습
    early_stopping = EarlyStopping(patience = 10)
    check_point = MyModelCheckpoint('best_model.keras', monitor = 'val_loss', mode = 'min', save_best_only = True, verbose = 1)

    hist = model.fit(T_data, T_label, epochs=100, batch_size=32, verbose=1, validation_data=(V_data, V_label), callbacks=[early_stopping, check_point])
    
    model.summary() 
    model_count += 1

# Arrhythmia Test
###############################################################################    
    Real_loss, Real_acc = model.evaluate({'main_input' : test_data}, {'arrhythmia': t_label})
    Real_pred = model.predict(test_data)
    Real_pred = Real_pred[:,0]
    

    for i in range(len(Real_pred)):
        if(0.5 <= Real_pred[i]):
            Real_pred[i]=1
        else:
            Real_pred[i]=0
            
            
    # confusion matrix 생성  
    t_label = t_label[:,0]
    
    te_conf_matrix = confusion_matrix(t_label, Real_pred)
    print(te_conf_matrix)
    print('\n')
    
    
    # confusion matrix를 이용한 지표
    tn, fp, fn, tp = te_conf_matrix.ravel()
    
    rows = (tp + fn) / (tn + fp + fn + tp)
    cols = (tn + fp) / (tn + fp + fn + tp)

    Real_pre = tp / (tp + fp)
    Real_rec = tp / (tp + fn)
    Real_spe = tn / (tn + fp)
    F_score = 2 * Real_spe * Real_rec / (Real_rec + Real_spe)
    
    print('\n')
    Real_result.append(Real_loss)
    Real_result.append(Real_acc) # accuracy(정확도): 맞는 것을 맞다, 틀린 것을 틀렸다 한 수치
    Real_result.append(Real_pre) # precision(정밀도): 예측의 quality 수치, 맞춘 것 중 관심 범주인 수치
    Real_result.append(Real_rec) # recall(재현도): 관심 범주 중 맞춘 수치
    Real_result.append(Real_spe) # specificity(특이도): 관심 범주가 아닌 것 중 맞춘 수치
    Real_result.append(F_score) # f-score

model_count = 0


print("-----------------------------Result----------------------")
print("Arrhythmia detection")
print(R_Arr_results)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras


from keras import models
from sklearn.metrics import confusion_matrix

model = models.load_model('best_model.keras')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Real_result=[]

# 테스트 라벨
test_label=np.zeros([1260,1])
for i in range(len(test_label)) :
    if i < 630 :
        test_label[i]=1

# 테스트 데이터 호출
test_data = np.load('./test_data.npy')
test_data = np.expand_dims(test_data, axis = -1)

# 라벨 제작
t_label = tf.keras.utils.to_categorical(test_label, num_classes=2)

# 모델에서 손실, 정확도 추출
Real_loss, Real_acc = model.evaluate({'main_input' : test_data}, {'arrhythmia': t_label}, verbose=2)

#모델에서 테스트 데이터에 대한 예측값 추출
Real_pred = model.predict(test_data)
Real_pred = Real_pred[:,0]
Real_pred = (Real_pred >= 0.5).astype(int)
            
# confusion matrix 생성  
t_label = t_label[:,0]

te_conf_matrix = confusion_matrix(t_label, Real_pred)

tn, fp, fn, tp = te_conf_matrix.ravel()
    
rows = (tp + fn) / (tn + fp + fn + tp)
cols = (tn + fp) / (tn + fp + fn + tp)

Real_pre = tp / (tp + fp)
Real_rec = tp / (tp + fn)
Real_spe = tn / (tn + fp)
F_score = 2 * Real_spe * Real_rec / (Real_rec + Real_spe)

print('\n')
Real_result.append(Real_loss) 
Real_result.append(Real_acc) # accuracy(정확도): 맞는 것을 맞다, 틀린 것을 틀렸다 한 수치
Real_result.append(Real_pre) # precision(정밀도): 예측의 quality 수치, 맞춘 것 중 관심 범주인 수치
Real_result.append(Real_rec) # recall(재현도): 관심 범주 중 맞춘 수치
Real_result.append(Real_spe) # specificity(특이도): 관심 범주가 아닌 것 중 맞춘 수치
Real_result.append(F_score) # f-score

print("-----------------------------Result----------------------")
print("Arrhythmia detection")
print(te_conf_matrix)
print('loss:', Real_loss)
print('acc:', Real_acc)
print('precision:', Real_pre)
print('recall:', Real_rec)
print('specificity:', Real_spe)
print('f-score', F_score)